# Simulated Annealing for Mars Lander

In [1]:
import math
import random

In [2]:
import peforth
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.26 (<class 'str'>)


In [2]:
# 改寫 for Mars Lander 乍看以為要頭痛一會兒，其實 Mars Lander 的 state/genome 很簡單，就是一串 (int,int) 
# pairs 而已。前面基礎討論有提到，以 SA 而言就是 mutate 某一個 gene 也就是 command pair 而已。但是考慮 Mars
# Lander 的限制條件，以 epidode-1 而言，command 的改變要做的決定是 -1 0 +1 其中之一才對，而非 0~4. 

# 長度的問題，
# 我想的是需要另一個 function 不夠時直接 copy 最後一個 gene 頂替即可。
# 會發現不夠的是 call fitness function 的人，那就是由它處理。

def get_random_neighbour(state):
    
    neighbour = state[:]  # Deep copy 利用 [:] slice copy, 若用 list(state) 不行！

    idx = random.randint(0,len(state)-1) # 選 1 個 gene 
    cmd = random.randint(-1, 1)  # -1, 0, +1 當中選 1 個 command 

    neighbour[idx][1] = cmd
    
    return neighbour

In [3]:
# ○ 成功著陸的，油料剩越多的越好。
# ○ 一個油料耗盡一個墜毀哪個好？墜毀時的速度減到+-40 以內需要多少油料？
# ○ 油料耗盡時的高度距離與速度以自由落體觸地，然後把速度調整到+-40 以內需要多少油料？
# ○ 由以上三點分析看來， cost_of_state(state) where state = ( step, x, [y], hspeed, [vspeed], [fuel], rotate, power ) 只參考 state 裡的三項 data. 
# ○ 根據我之前對 y=100 的研究 該 state 來自 log 的最後一筆 state. Simulation 時，該 state 就是 y 首次跨越 flat ground 高度的該 step, 也就是觸地著陸之時. 這個 cost_of_state() 不用管，它只管算出分數即可。
# ○ cost 以 0 為目標。Score 用油料剩餘量計算，由 fuel0 扣下來可到 0

def cost_of_state(state):
    
    
    cost = fuel0  # 初始油料
    # 安全著陸：
        
    COL = 0
    NAT = 1
    BEV = 2
    CIG = 3
    ANI = 4

    lost = 15 
    
    # According to the recent state, build a hash table.
    # Its function is to convert a token e.g. 'red', 'dog', 'dunhill', ... etc into location e.g. (house,attribute)
    search = {}
    for i in range(5):
        for j in range(5):
            search[state[i][j]] = (i,j);
    
    # Get the house that contains the token. Return a row of the state 
    # which is something like this: ['yellow', 'norwegian', 'root root beer', 'blue master', 'bird']
    def house(token):
        # 找到 (i,j) 傳回 state 之橫的一欄，就是符合條件的某 house. 
        # return [state[search(token)[0]][j] for j in range(5)]
        return state[search[token][0]]

    clue = [];
    # 1. 英國人住在紅房子裡
    clue.append(int(house('brit')[COL]=='red'));
    # 2. 瑞典人養了一條狗    
    clue.append(int(house('swede')[ANI]=='dog'));
    # 3. 丹麥人喝茶    
    clue.append(int(house('dane')[BEV]=='tea'));
    # 4. 綠房子在白房子左邊    
    a = search['green'][0];  # 左到右 house 的順位
    b = search['white'][0];  # 左到右 house 的順位
    clue.append(int(a < b));
    # 5. 綠房子主人喝咖啡    
    clue.append(int(house('green')[BEV]=='coffee'));
    # 6. 抽pall mall煙的人養了一隻鳥    
    clue.append(int(house('pall mall')[ANI]=='bird'));   
    # 7. 黃房子主人抽dunhill煙    
    clue.append(int(house('yellow')[CIG]=='dunhill'));   
    # 8. 住在中間那間房子的人喝牛奶
    clue.append(int(state[2][BEV]=='milk'));
    # 9. 挪威人住在第一間房子    
    clue.append(int(state[0][NAT]=='norwegian'));
    # 10. 抽混合菸的人住在養貓人的旁邊     
    a = search['blends'][0];   # 左到右 house 的順位
    b = search['cat'][0];      # 左到右 house 的順位
    clue.append(int(abs(a-b)==1));
    # 11. 養馬人住在dunhill煙的人旁邊     
    a = search['horse'][0];    # 左到右 house 的順位
    b = search['dunhill'][0];  # 左到右 house 的順位
    clue.append(int(abs(a-b)==1));
    # 12. 抽blue master煙的人喝啤酒     
    clue.append(int(house('blue master')[BEV]=='root beer'));   
    # 13. 德國人抽prince煙     
    clue.append(int(house('german')[CIG]=='prince'));   
    # 14. 挪威人住在藍房子旁邊     
    a = search['norwegian'][0];  # 左到右 house 的順位
    b = search['blue'][0];       # 左到右 house 的順位
    clue.append(int(abs(a-b)==1));
    # 15. 抽混合菸的人的鄰居喝礦泉水     
    a = search['blends'][0];    # 左到右 house 的順位
    b = search['water'][0];     # 左到右 house 的順位
    clue.append(int(abs(a-b)==1));
    
    return lost - sum(clue)


In [4]:
# fitness function 
# https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
def sa(initial):
    current = initial
    current_cost = cost_of_state(current)
    temp = 1.0  # <---- [ ] Hyperparameter
    num_iterations = 0

    while current_cost > 0:
        neighbour = get_random_neighbour(current)
        neighbour_cost = cost_of_state(neighbour)

        cost_delta = neighbour_cost - current_cost

        if cost_delta <= 0 or random.random() < math.exp(-cost_delta/temp):
            current, current_cost = neighbour, neighbour_cost

        num_iterations += 1
        if num_iterations % 500 == 0 and temp > 0.20:  # <---- [ ] Hyperparameter
            temp -= 0.05  # <---- [ ] Hyperparameter

    return current, num_iterations

In [5]:
def epoch(state=None):
    nationalities = [ 'dane',      'brit',   'swede',       'norwegian', 'german'    ]
    colours       = [ 'yellow',    'red',    'white',       'green',     'blue'      ]
    animals       = [ 'horse',     'cat',    'bird',        'fish',      'dog'       ]
    beverages     = [ 'water',     'tea',    'milk',        'coffee',    'root beer' ]
    cigars        = [ 'pall mall', 'prince', 'blue master', 'dunhill',   'blends'    ]

    random.shuffle(nationalities)
    random.shuffle(colours      )
    random.shuffle(animals      )
    random.shuffle(beverages    )
    random.shuffle(cigars       )
    
    attributes = [colours, nationalities, beverages, cigars, animals]

    NUM_HOUSES = 5
    initial = []
    s = '['
    
    if state: 
        initial = state; 
    else:
        for i in range(NUM_HOUSES):
            initial.append([attr[i] for attr in attributes])
    
    
    # random.seed(100)

    solution, iterations = sa(initial)

    for house in solution:
        s += str(house) + ',';
    s = s[:-1] + ']';
    return s,iterations

In [6]:
if __name__ == "__main__":
    ss = '';
    for i in range(10000):
        s, iter = epoch()
        ss += "{} Number of iterations:{}\n".format(s, iter)
        print('.', end="")
    %f ss char results.txt writeTextFile 
    %f .( Done !!! ) cr

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

http://localhost:8888/lab/workspaces/auto-m/tree/OneDrive/%E6%96%87%E4%BB%B6/Jupyter%20Notebooks/I%20study%20Pandas.ipynb

In [7]:
import pandas as pd

## 研究 pandas read data 的眾多方法

我看其中就是 pd.read_clipboard() 好用。先把資料抓進 clipboard 然後執行之即得。
研究 iterations 數的分布的目的是想藉以調整 Annealing Algorithm 的 hyper-parameter 

In [8]:
data = pd.read_clipboard()
data

,Data
0,8910
1,9532
2,6423
3,6776
4,22155
...,...
995,13802
996,6385
997,7525
998,8103


In [41]:
%f data :> describe() ==>
%f data :> Data.median() --> 

data :> describe() ==>
                 Data
count    1000.000000
mean    13933.352000
std     13448.507494
min      1242.000000
25%      6925.250000
50%      8452.000000
75%     15068.000000
max    113487.000000 (<class 'pandas.core.frame.DataFrame'>)
data :> Data.median() --> 8452.0 (<class 'numpy.float64'>)


In [45]:
data = pd.read_clipboard()
data

,Iterations
0,4851
1,6097
2,20168
3,10302
4,27273
...,...
9995,6986
9996,9056
9997,18983
9998,7096


In [47]:
%f data :> describe() ==>
%f data :> Iterations.median() --> 

data :> describe() ==>
          Iterations
count   10000.00000
mean    13178.39830
std     12065.95423
min        45.00000
25%      6995.75000
50%      8426.00000
75%     14400.25000
max    152908.00000 (<class 'pandas.core.frame.DataFrame'>)
data :> Iterations.median() --> 8426.0 (<class 'numpy.float64'>)
